In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[11],9
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004}"
nv,-,1000
tsfc,257,257


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.001222                    1  0.098346
0.000750        2 -0.156257                    2 -0.131744
0.001052        3 -0.113086                    3 -0.094381
0.001476        4 -0.091351                    4 -0.078507
0.002070        5 -0.074332                    5 -0.076595
0.002904        6 -0.057947                    6 -0.080171
0.004074        7 -0.041303                    7 -0.082350
0.005714        8 -0.024418                    8 -0.072045
0.008015        9 -0.007762                    9 -0.053149
0.011243       10  0.008256                   10 -0.026693
0.015771       11  0.023563                   11  0.005482
0.022122       12  0.039150                   12  0.042281
0.031031       13  0.060655                   13  0.086773
0.043528       14  0.076175                   14  0.122284
0.061057       15  0.073413                   15  0.132771
0.085645       16  0.068762                   16  0.128323
0.120136       17  0.070392                   17  0.116508
0.168516       18  0.075295                   18  0.098854
0.236378       19  0.083674                   19  0.081265
0.331549       20  0.098614                   20  0.076522
0.465100       21  0.134445                   21  0.107388
0.652400       22  0.121463                   22  0.097394
0.915100       23  0.063170                   23  0.051540
1.283650       24  0.030823                   24  0.035190
1.800600       25  0.016823                   25  0.027779
2.525700       26  0.009029                   26  0.019522
3.542800       27  0.003926                   27  0.010563
4.969550       28  0.001656                   28  0.003328
6.970850       29  0.001418                   29  0.000439
9.778100       30  0.001198                   30  0.000739
13.715850      31  0.000618                   31  0.001399
19.239350      32 -0.000109                   32  0.000891
26.987250      33 -0.000258                   33  0.000434
37.855300      34  0.000047                   34  0.000318
53.100050      35  0.000133                   35  0.000025
73.887500      36  0.000160                   36 -0.000091
97.662500      37  0.000183                   37 -0.000047
121.437500     38  0.000042                   38 -0.000177
145.212500     39 -0.000144                   39 -0.000363
168.987500     40 -0.000257                   40 -0.000460
192.762500     41 -0.000365                   41 -0.000570
216.537500     42 -0.000483                   42 -0.000702
240.312500     43 -0.000623                   43 -0.000854
264.087500     44 -0.000840                   44 -0.001078
287.862500     45 -0.001303                   45 -0.001582
311.637500     46 -0.001621                   46 -0.001925
335.412500     47 -0.001419                   47 -0.001744
359.187500     48 -0.001116                   48 -0.001470
382.962500     49 -0.000943                   49 -0.001348
406.737500     50 -0.000749                   50 -0.001186
430.512500     51 -0.000507                   51 -0.000921
454.287500     52 -0.000205                   52 -0.000604
478.062500     53  0.000169                   53 -0.000158
501.837500     54  0.000622                   54  0.000258
525.612500     55  0.001159                   55  0.000686
549.387500     56  0.001774                   56  0.001166
573.162500     57  0.002456                   57  0.001767
596.937500     58  0.003183                   58  0.002530
620.712500     59  0.003924                   59  0.003315
644.487500     60  0.004649                   60  0.004314
668.262500     61  0.005332                   61  0.005196
692.037500     62  0.005958                   62  0.006036
715.812500     63  0.006531                   63  0.006801
739.587500     64  0.007074                   64  0.007502
763.362500     65  0.007625                   65  0.008173
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -1.164942  0.000000 -1.164942 -1.146064  8.379483e-09   
0.000624    2     -1.164957  0.000016 -1.164942 -1.146087  3.013670e-05   
0.000876    3     -1.164966  0.000019 -1.164946 -1.146098  3.765899e-05   
0.001229    4     -1.164976  0.000025 -1.164951 -1.146113  4.874139e-05   
0.001723    5     -1.164991  0.000034 -1.164956 -1.146133  6.380974e-05   
0.002417    6     -1.165010  0.000048 -1.164963 -1.146158  8.307933e-05   
0.003391    7     -1.165036  0.000067 -1.164969 -1.146191  1.068976e-04   
0.004757    8     -1.165069  0.000093 -1.164976 -1.146234  1.365930e-04   
0.006672    9     -1.165113  0.000131 -1.164982 -1.146290  1.756588e-04   
0.009359    10    -1.165169  0.000185 -1.164984 -1.146360  2.293473e-04   
0.013128    11    -1.165241  0.000261 -1.164980 -1.146449  3.061664e-04   
0.018415    12    -1.165331  0.000366 -1.164966 -1.146559  4.193467e-04   
0.025830    13    -1.165441  0.000510 -1.164931 -1.146691  5.888749e-04   
0.036232    14    -1.165564  0.000708 -1.164856 -1.146839  8.435249e-04   
0.050823    15    -1.165688  0.000963 -1.164725 -1.146993  1.209001e-03   
0.071291    16    -1.165818  0.001271 -1.164547 -1.147161  1.698530e-03   
0.100000    17    -1.165961  0.001649 -1.164313 -1.147350  2.323915e-03   
0.140271    18    -1.166111  0.002134 -1.163977 -1.147554  3.084354e-03   
0.196760    19    -1.166246  0.002773 -1.163473 -1.147759  3.950573e-03   
0.275997    20    -1.166317  0.003629 -1.162688 -1.147931  4.885090e-03   
0.387100    21    -1.166185  0.004795 -1.161390 -1.147940  5.902005e-03   
0.543100    22    -1.165267  0.006361 -1.158906 -1.147123  7.069288e-03   
0.761700    23    -1.163442  0.007682 -1.155760 -1.145294  7.763130e-03   
1.068500    24    -1.161537  0.008072 -1.153465 -1.143374  7.715718e-03   
1.498800    25    -1.159850  0.007956 -1.151893 -1.141657  7.792435e-03   
2.102400    26    -1.158361  0.007670 -1.150691 -1.140091  8.213102e-03   
2.949000    27    -1.157071  0.007286 -1.149785 -1.138669  8.749192e-03   
4.136600    28    -1.156061  0.006828 -1.149233 -1.137473  9.039051e-03   
5.802500    29    -1.155338  0.006432 -1.148906 -1.136549  8.771902e-03   
8.139200    30    -1.154754  0.006240 -1.148514 -1.135746  8.090673e-03   
11.417000   31    -1.154242  0.006194 -1.148048 -1.134990  7.620988e-03   
16.014700   32    -1.153887  0.006175 -1.147712 -1.134420  7.813206e-03   
22.464000   33    -1.153979  0.006184 -1.147795 -1.134445  8.518649e-03   
31.510500   34    -1.154657  0.006586 -1.148071 -1.135202  9.741067e-03   
44.200100   35    -1.155709  0.007708 -1.148001 -1.136408  1.142519e-02   
62.000000   36    -1.157106  0.009387 -1.147719 -1.138034  1.310378e-02   
85.775000   37    -1.158742  0.011473 -1.147269 -1.139942  1.475612e-02   
109.550000  38    -1.160111  0.013359 -1.146752 -1.141582  1.626275e-02   
133.325000  39    -1.161391  0.014758 -1.146634 -1.143165  1.734645e-02   
157.100000  40    -1.162856  0.015815 -1.147040 -1.145044  1.820250e-02   
180.875000  41    -1.164537  0.016774 -1.147763 -1.147257  1.911828e-02   
204.650000  42    -1.166472  0.017680 -1.148793 -1.149835  2.009028e-02   
228.425000  43    -1.168695  0.018541 -1.150154 -1.152820  2.109906e-02   
252.200000  44    -1.171278  0.019370 -1.151908 -1.156257  2.213107e-02   
275.975000  45    -1.174458  0.020186 -1.154273 -1.160349  2.318805e-02   
299.750000  46    -1.179076  0.021133 -1.157943 -1.166038  2.442024e-02   
323.525000  47    -1.185636  0.023129 -1.162507 -1.173876  2.683737e-02   
347.300000  48    -1.193548  0.027045 -1.166503 -1.183224  3.127413e-02   
371.075000  49    -1.202276  0.032629 -1.169647 -1.193447  3.735543e-02   
394.850000  50    -1.212061  0.039760 -1.172301 -1.204723  4.483462e-02   
418.625000  51    -1.223145  0.048735 -1.174410 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')